In [18]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


### Seguimiento con frames leidos desde la webcam

In [17]:
# Condición de terminación: 10 iteraciones o moverse al menos 1pt
#----------------------------------------------------------------
term_crit = ( cv.TERM_CRITERIA_COUNT | cv.TERM_CRITERIA_EPS, 10, 1 )
cap = cv.VideoCapture(0)
selected_roi = False
show_track = True

centers = list()
while(1):

    ret ,frame = cap.read()

    if ret:
        
        if selected_roi:
            
            hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
            dst = cv.calcBackProject([hsv], [0], roi_hist, [0,180], 1)
            ret, track_window = cv.meanShift(dst, track_window, term_crit)
            x, y, w, h = track_window
            
            # recalculamos las caracteristicas para actualizar nuestro descriptor
            # hsv_roi =  hsv[y:y+h, x:x+w,:]
            # mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
            # roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
            # cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)
            
            frame = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
            
            if show_track:
            
                centers.append((int(x + w/2), int(y + h/2)))
                for center in centers:
                    cv.circle(frame, center, radius=3, color=(255,0,0),thickness = -1)
                
        cv.imshow('Seguimiento',frame)
        key = cv.waitKey(50) & 0xFF
        # Usamos funciones de GUI de OpenCV para seleccionar una ROI
        if key == ord("s"):
            track_window = cv.selectROI("Seguimiento", frame, fromCenter=False, showCrosshair=False)
            x, y, w, h = track_window
            selected_roi = True
            # caracteristicas para meanshift
            hsv_roi =  cv.cvtColor(frame[y:y+h, x:x+w,:], cv.COLOR_BGR2HSV)
            mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
            roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
            cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)
            
        elif key == ord("q"):
            break
    else:
        break

cv.destroyAllWindows()
cap.release()